<a href="https://colab.research.google.com/github/CJL-INVEST/invest/blob/main/dual_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bt
!pip install finterstellar
!pip install quantstats
!pip install finance-datareader
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import bt
import finterstellar as fs
import quantstats as qs
import pandas as pd
from datetime import datetime, timedelta
import math
import FinanceDataReader as fdr
import yfinance as yf
%matplotlib inline

In [ ]:
ticker = ['SPY', 'EFA','BIL','AGG']
start_day = datetime(2000,1,1)
end_day = datetime.now()
# end_day = datetime(2019,8,30)

In [ ]:
df =  yf.download(ticker, start=start_day)['Adj Close']
df = df.dropna()

[*********************100%***********************]  4 of 4 completed


In [ ]:
df

,AGG,BIL,EFA,SPY
Date,,,,
2007-05-30,63.047546,82.658524,50.423717,112.702774
2007-05-31,62.907959,82.658524,50.673878,112.585297
2007-06-01,62.842991,82.676521,51.005322,113.143349
2007-06-04,62.919453,82.694557,51.142899,113.158028
2007-06-05,62.741047,82.694557,50.880245,112.710106
...,...,...,...,...
2022-12-14,99.690002,91.299057,67.730003,397.574341
2022-12-15,99.610001,91.319000,66.000000,387.848999
2022-12-16,99.349998,91.319000,65.419998,383.269989


In [ ]:
df = df.resample('M').last()

In [ ]:
# 모멘텀 지수 계산 전체 함수

def get_momentum(x):
  temp_list = [0 for i in range(len(x.index))]
  momentum = pd.Series(temp_list, index=x.index)
  try: 
    momentum = df[x.name-timedelta(days=370):x.name-timedelta(days=365)].iloc[-1][ticker]    
  except Exception as e:
    # print('Error: ', str(e))
    pass
  
  return momentum 

In [ ]:
# df_mom = df.copy()
# df_mom[ticker] = df[ticker].apply(lambda x: get_momentum(x), axis=1)

In [ ]:
# df_mom

In [ ]:
df_mon12 = df.pct_change(periods=12)
df_mon12.head(13)

,AGG,BIL,EFA,SPY
Date,,,,
2007-05-31,NaN,NaN,NaN,NaN
2007-06-30,NaN,NaN,NaN,NaN
2007-07-31,NaN,NaN,NaN,NaN
2007-08-31,NaN,NaN,NaN,NaN
2007-09-30,NaN,NaN,NaN,NaN
2007-10-31,NaN,NaN,NaN,NaN
2007-11-30,NaN,NaN,NaN,NaN
2007-12-31,NaN,NaN,NaN,NaN
2008-01-31,NaN,NaN,NaN,NaN


In [ ]:
df_mon12_slected = pd.DataFrame(columns=ticker)

In [ ]:
def select_item(x):
  asset = None
  temp_list = [False for i in range(len(x.index))]
  selected_info = pd.Series(temp_list, index=x.index)
  if df_mon12['SPY'].loc[x.name] < df_mon12['BIL'].loc[x.name]:
      asset = 'AGG'
  else:
    if df_mon12['EFA'].loc[x.name] > df_mon12['SPY'].loc[x.name]:
      asset = 'EFA'
    else:
      asset = 'SPY'
  selected_info[asset] = True
  # print('---')
  # print(selected_info)
  return selected_info


In [ ]:
df_mon12_slected[ticker] = df_mon12[ticker].apply(lambda x: select_item(x), axis=1)

In [ ]:
df_mon12_slected

,SPY,EFA,BIL,AGG
Date,,,,
2007-05-31,True,False,False,False
2007-06-30,True,False,False,False
2007-07-31,True,False,False,False
2007-08-31,True,False,False,False
2007-09-30,True,False,False,False
...,...,...,...,...
2022-08-31,False,False,False,True
2022-09-30,False,False,False,True
2022-10-31,False,False,False,True


In [ ]:
initial_capital = 100000.00

In [ ]:
st_original_dual_momentum = bt.Strategy('Original_Dual_Momentum',
                     [
                      bt.algos.RunAfterDate(start_day),
                      bt.algos.RunMonthly(run_on_end_of_period=True),
                      bt.algos.PrintDate(),
                      bt.algos.PrintInfo('{name}:{now}. Value:{_value:0.0f}, Price:{_price:0.4f}'),
                      bt.algos.SelectWhere(df_mon12_slected),
                      bt.algos.WeighEqually(),
                      bt.algos.PrintTempData(),
                      bt.algos.Rebalance()
                     ])

bt_original_dual_momentum = bt.Backtest(st_original_dual_momentum, df, initial_capital=initial_capital)

In [ ]:
ret = bt.run(bt_original_dual_momentum)

2007-05-31 00:00:00
Original_Dual_Momentum:2007-05-31 00:00:00. Value:100000, Price:100.0000
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-06-30 00:00:00
Original_Dual_Momentum:2007-06-30 00:00:00. Value:98538, Price:98.5383
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-07-31 00:00:00
Original_Dual_Momentum:2007-07-31 00:00:00. Value:95454, Price:95.4537
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-08-31 00:00:00
Original_Dual_Momentum:2007-08-31 00:00:00. Value:96678, Price:96.6784
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-09-30 00:00:00
Original_Dual_Momentum:2007-09-30 00:00:00. Value:100420, Price:100.4203
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-10-31 00:00:00
Original_Dual_Momentum:2007-10-31 00:00:00. Value:101782, Price:101.7824
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-11-30 00:00:00
Original_Dual_Momentum:2007-11-30 00:00:00. Value:97841, Price:97.8410
{'selected': ['SPY'], 'weights': {'SPY': 1.0}}
2007-12-31 00:00:00
Original

In [ ]:
ret.display()

Stat                 Original_Dual_Momentum
-------------------  ------------------------
Start                2007-05-30
End                  2022-12-31
Risk-free rate       0.00%

Total Return         146.50%
Daily Sharpe         2.41
Daily Sortino        4.05
CAGR                 5.96%
Max Drawdown         -19.43%
Calmar Ratio         0.31

MTD                  0.24%
3m                   2.72%
6m                   -2.11%
YTD                  -15.94%
1Y                   -15.94%
3Y (ann.)            3.55%
5Y (ann.)            4.00%
10Y (ann.)           6.76%
Since Incep. (ann.)  5.96%

Daily Sharpe         2.41
Daily Sortino        4.05
Daily Mean (ann.)    137.58%
Daily Vol (ann.)     57.20%
Daily Skew           -0.37
Daily Kurt           1.05
Best Day             10.91%
Worst Day            -12.48%

Monthly Sharpe       0.53
Monthly Sortino      0.89
Monthly Mean (ann.)  6.59%
Monthly Vol (ann.)   12.51%
Monthly Skew         -0.37
Monthly Kurt         1.03
Best Month           10.9